In [ ]:
! pip install transformers torch sentencepiece accelerate

In [4]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Load and preprocess the dataset
file_path = 'quantized_coordinates.csv'
coordinates_df = pd.read_csv(file_path)
coordinates_df['sequence'] = coordinates_df['y_quant'].astype(str) + ' ' + coordinates_df['x_quant'].astype(str)

# Prepare input-output pairs for training
def prepare_data(df, input_len=3):
    input_sequences = []
    output_sequences = []
    for i in range(len(df) - input_len):
        input_seq = ' '.join(df['sequence'].iloc[i:i+input_len])
        output_seq = ' '.join(df['sequence'].iloc[i:i+input_len+1])
        input_sequences.append(input_seq)
        output_sequences.append(output_seq)
    return input_sequences, output_sequences

input_seqs, output_seqs = prepare_data(coordinates_df)

# Split the data into train and validation sets (80% train, 20% validation)
train_inputs, val_inputs, train_outputs, val_outputs = train_test_split(
    input_seqs, output_seqs, test_size=0.2, random_state=42
)

# Custom Dataset Class
class CoordinateDataset(Dataset):
    def __init__(self, inputs, outputs, tokenizer, max_len, device):
        self.inputs = inputs
        self.outputs = outputs
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.device = device

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_seq = self.inputs[idx]
        output_seq = self.outputs[idx]
        
        inputs = self.tokenizer(input_seq, max_length=self.max_len, padding='max_length', truncation=True, return_tensors="pt")
        outputs = self.tokenizer(output_seq, max_length=self.max_len, padding='max_length', truncation=True, return_tensors="pt")
        
        return {
            'input_ids': inputs.input_ids.flatten().to(self.device),  # Move tensors to MPS
            'attention_mask': inputs.attention_mask.flatten().to(self.device),  # Move tensors to MPS
            'labels': outputs.input_ids.flatten().to(self.device)  # Move tensors to MPS
        }

# Initialize model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Check if MPS is available and move model to MPS
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model = model.to(device)

# Dataset parameters
MAX_LEN = 50
BATCH_SIZE = 8

# Create train and eval datasets
train_dataset = CoordinateDataset(train_inputs, train_outputs, tokenizer, max_len=MAX_LEN, device=device)
eval_dataset = CoordinateDataset(val_inputs, val_outputs, tokenizer, max_len=MAX_LEN, device=device)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",  # Evaluate every X steps
    eval_steps=50,  # Evaluation every 50 steps
    save_steps=500,  # Save checkpoint every 500 steps
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Inference: Generate a sequence given the first few points
def generate_sequence(model, tokenizer, input_sequence, max_length=50):
    inputs = tokenizer(input_sequence, return_tensors="pt").to(device)
    output = model.generate(inputs.input_ids, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)




/Users/naveenmalla/Documents/Projects/Thesis/Code/Synthetic_City_Generator/venv/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  1%|          | 10/1470 [00:03<08:00,  3.04it/s]

{'loss': 9.3745, 'grad_norm': 168.86508178710938, 'learning_rate': 4.965986394557823e-05, 'epoch': 0.07}


  1%|▏         | 20/1470 [00:07<08:56,  2.71it/s]

{'loss': 4.251, 'grad_norm': 41.80757522583008, 'learning_rate': 4.931972789115647e-05, 'epoch': 0.14}


  2%|▏         | 30/1470 [00:11<09:00,  2.66it/s]

...

{'loss': 0.1896, 'grad_norm': 0.8950478434562683, 'learning_rate': 3.4013605442176873e-07, 'epoch': 9.93}


100%|██████████| 1470/1470 [16:58<00:00,  1.51it/s]

{'loss': 0.1813, 'grad_norm': 0.5268719792366028, 'learning_rate': 0.0, 'epoch': 10.0}


100%|██████████| 1470/1470 [17:03<00:00,  1.44it/s]


{'train_runtime': 1023.446, 'train_samples_per_second': 11.432, 'train_steps_per_second': 1.436, 'train_loss': 0.3312650464018997, 'epoch': 10.0}
Input: 88 121 89 121 90
Predicted Sequence: 88 121 89 121 90 88 121


In [6]:
# Example prediction
input_sequence = '80 118 79 123 88 127'
predicted_sequence = generate_sequence(model, tokenizer, input_sequence)
print(f"Input: {input_sequence}")
print(f"Predicted Sequence: {predicted_sequence}")

Input: 80 118 79 123 88 127
Predicted Sequence: 80 118 79 123 88 127 88 127


In [3]:
import torch
print(torch.backends.mps.is_available())  # Should return True on M1 Mac
print(torch.backends.mps.is_built())      # Should return True if PyTorch was built with MPS support


True
True
